In [1]:
library(jsonlite)
library(rlist)
library(data.table)
suppressPackageStartupMessages(library("dplyr"))
library(XML)
library(methods)

In [2]:
# We select the xmls with the datas we need
setwd('/home/jovyan/xml/')


In [3]:
suppressWarnings({
    options(stringsAsFactors = FALSE)
# Open different xml in the same time
# We select the xmls with the datas we need
xmlfiles=list.files(pattern='product3')
# We remove the datas about genetic disorders, they have no interest here
xmlfiles=xmlfiles[!xmlfiles=='en_product3_156.xml']
# We put the 35 xml files from  orphadata in data frames
indice_list=1
xmldoc=list()
products3=list()
    for (i in 1:length(xmlfiles)) 
{
  xmldoc[[indice_list]]=xmlParse(xmlfiles[i]) 
  products3[[indice_list]]=xmlToList(xmldoc[[indice_list]])
  indice_list=indice_list+1
}
    
})

In [4]:
# We get the Classification information
prods3=list()
for (i in 1:length(products3))
{
  prods3[[i]]=unlist(products3[[i]]$ClassificationList)
}
# Each element in the list is a disorder

In [5]:
# We get the OrphaCode and the DisorderType for each disorder
disordertype = list()
OrphaCode = list()
indice_orpha = 1
indice_disordertype = 1
for (i in 1:length(prods3))
  for (j in 1:length(prods3[[i]])) 
    if (grepl("Disorder.OrphaCode", names(prods3[[i]][j]))) {
      OrphaCode[[indice_orpha]] = data.frame(classification = prods3[[i]][2],
                                             OrphaCode = prods3[[i]][j])
      indice_orpha = indice_orpha + 1
    } else if (grepl("DisorderType..attrs.id", names(prods3[[i]][j]))) {
      disordertype[[indice_disordertype]] = data.frame(classification = prods3[[i]][2],
                                                       disordertype = prods3[[i]][j])
      indice_disordertype = indice_disordertype + 1
    } 

disordertype = rbindlist(disordertype)
OrphaCode = rbindlist(OrphaCode)

In [6]:
# We only keep Disorders
disorders = cbind.data.frame(OrphaCode, disordertype = disordertype$disordertype)

disorders = filter(disorders, !duplicated(disorders))
disorders = filter(disorders, disordertype == "21429" |
                     disordertype == "21401" |
                     disordertype == "21415" |
                     disordertype == "21394" |
                     disordertype == "21422" |
                     disordertype == "21408")

In [7]:
# We count how much times an OrphaCode is duplicated
table_count=as.data.frame(table(disorders$OrphaCode))
names(table_count)=c('OrphaCode','Number of classifications')
table_count=filter(table_count, `Number of classifications` > 0)

In [8]:
# We look for ORPHAcodes which are duplicated at least once
indice_list=1
duplicates=list()
for (i in 1:nrow(table_count))
  if (table_count$`Number of classifications`[i] > 1)
  {
    duplicates[[indice_list]]=table_count[i, ]
    indice_list=indice_list+1
  }
duplicates=rbindlist(duplicates,use.names = TRUE)

In [9]:
# Multiclassified disorders
number_of_disorders_multiclassified=nrow(duplicates)
percentage_of_number_of_disorders_multiclassified=number_of_disorders_multiclassified/nrow(table_count)*100
print(paste0("The number of disorders multiclassified is ", number_of_disorders_multiclassified))
print(paste0("The percentage of disorders multiclassified is ", round(percentage_of_number_of_disorders_multiclassified, digits = 2), "%"))

round(percentage_aligned_disorders_with_OMIM, digits = 2)

[1] "The number of disorders multiclassified is 4186"
[1] "The percentage of disorders multiclassified is 67.3206818912834%"


In [10]:
# Uniclassified disorders
number_of_disorders_uniclassified=nrow(table_count) - number_of_disorders_multiclassified
percentage_of_number_of_disorders_uniclassified=number_of_disorders_uniclassified/nrow(table_count)*100
print(paste0("The number of disorders uniclassified is ", number_of_disorders_uniclassified))
print(paste0("The percentage of disorders uniclassified is ", round(percentage_of_number_of_disorders_uniclassified, digits = 2), "%"))


[1] "The number of disorders uniclassified is 2032"
[1] "The percentage of disorders uniclassified is 32.6793181087166%"


In [11]:
### Stats : min/max, mean, median of the number of classifications for the disorders
summary(table_count$`Number of classifications`)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   2.000   2.303   3.000  13.000 